# Model Training - CNN


#### Here we train different CNN models from scratch to detect pneumonia

#### In this notebook, we do the following :-
1. Define basic parameters for image processing like image_size and batch_size<br>
2. Create a tensorflow data pipeline to efficiently read the images while training the model<br>
3. Train three different <b>5-layer-deep</b> CNN model with <b>Global Max Pooling</b>, <b>Global Average Pooling</b> and <b>Flatten</b> layer respectively to see which flattening layer gives the best performance on AUC<br>
4. Train <b>3-layer-deep</b> and <b>7-layer-deep</b> CNN model with GlobalMaxPooling layer to observe the effect of layer depth on model performance<br>

<i>The performance analysis of all these trained models is done in "d) Model Performance Analysis" notebook</i><br>

#### The data pipeline does the following :-
1. Reads a "batch" of image file_paths from the metadata
2. Read those images and processes it - resizing, casting into appropiate datatype, image_augmentation (only in training)
4. Return the batch of processed images for feeding into the model for training or validation
5. The data pipeline shuffles the training set while creating a batch to train the model to avoid ordering bias
6. It also automatically determines the appropiate #parallel_threads while mapping the file path to output image to save time
7. It prefetches the next batch while the current batch is feed into the model for training to save time


#### Following relevant layers were used in the architecture of the model
1. <b>Conv2D</b> layer with 3x3 filters - to apply convolution on the input image. We have kept the 'same' padding to reduce the size of the image
2. <b>BatchNormalization</b> - to improve training using regularization and reducing internal covariate shift while training.
3. <b>ReLU</b> - to add non-linaer activation to convoluted images for learning robust features
4. <b>MaxPooling2D</b> to reduce the size of the image and at the same time keep prominent features intact!
5. <b>GlobalMaxPooling2D</b> - this layer reduces the feature maps to a singular values by taking maximum of the most prominent features within those maps for feeding into fully connected layers
6. <b>GlobalAveragePooling2D</b> - this layer reduces the feature maps to a singular values by taking the average of all values those maps for feeding into fully connected layers
7. <b>Dropout</b> - this is used in fully connected layers to avoid overfitting by learning robust features

#### Following callback methods were used while training :-
1. <b>ReduceLROnPlateau</b> - this will reduce the learning rate by some factor if the monitored metric (val_loss) decreases twice in epochs while training. This will avoid overfitting
2. <b>ModelCheckpoint</b> - this saves the model after each epoch. This will allow us to use any intermediate epoch model if needed
3. <b>CSVLogger</b> - this saves the model performance metrics after each epoch in a csv file for later analysis
4. <b>EarlyStopping</b> - this will stop training of the model if monitored metric (val_loss) decreases 3 times in a row. This will avoid overfitting

# Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, GlobalMaxPooling2D, ReLU, GlobalAveragePooling2D
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, EarlyStopping


# Parameters for Image Processing

In [2]:
downsize_ratio = 4
IMG_SIZE = int(1024 / downsize_ratio)
batch_size = 12
print(f'Image size for training the model will be {IMG_SIZE} x {IMG_SIZE}')

Image size for training the model will be 256 x 256


Experiment with other image sizes like (512 x 512) and (341 x 341) was done. In that case, the training only gets computationally expensive while not leading to significant increment in model performance.

# Crate Data Pipeline

### Prepare metadata for images

In [5]:
train_df = pd.read_csv('Data/pneumonia/train_metadata.csv')
train_df['file_path'] = 'Data/pneumonia/Training/Images/' + train_df['patientId'] + '.png'

train_data = list(zip(train_df['file_path'], train_df['Target']))
train_paths, train_target = zip(*train_data)

validation_df = pd.read_csv('Data/pneumonia/val_metadata.csv')
validation_df['file_path'] = 'Data/pneumonia/Training/Images/' + validation_df['patientId'] + '.png'

validation_data = list(zip(validation_df['file_path'], validation_df['Target']))
validation_paths, validation_target = zip(*validation_data)

print(train_df.shape)
print(validation_df.shape)

(25727, 12)
(2250, 12)


### Mapping Function to Process Image

In [ ]:
# Data Augmentation block
from tensorflow.keras.layers import RandomFlip

augmentation_block = tf.keras.Sequential([
    RandomFlip("vertical"),
], name="data_augmentation")


In [7]:
def load_and_process_train(image_path, target):

    #read file
    image = tf.io.read_file(image_path)

    #process image
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE], method = 'bilinear')
    image = tf.cast(image, tf.float32)

    #augmentation for training
    image = augmentation_block(image, training=True)
    
    return image, target

def load_and_process_validation(image_path, target):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE], method = 'bilinear')
    image = tf.cast(image, tf.float32)
    return image, target


### Create Tensorflow Dataset

In [8]:
# TF datasets
train_ds = tf.data.Dataset.from_tensor_slices((list(train_paths), list(train_target)))
train_ds = train_ds.map(load_and_process_train, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(100).batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((list(validation_paths),  list(validation_target)))
val_ds = val_ds.map(load_and_process_validation, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)


# Modelling

# Train 5 Layer CNN Model with different flattening layers

In [9]:
#CNN Layers
def convolutional_block_5layer(inputs):
    
    #1
    x = Conv2D(60, (3,3), padding = 'same')(inputs) #254 x 254 x 60 
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2,2))(x) #127 x 127 x 60
    
    #2
    x = Conv2D(120, (3,3), padding = 'same')(x) #125 x 125 x 120
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2,2))(x) #62 x 62 x 120

    #3
    x = Conv2D(240, (3,3), padding = 'same')(x) #60 x 60 x 240
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2,2))(x) #30 x 30 x 240

    #4
    x = Conv2D(480, (3,3),  padding = 'same')(x) #28 x 28 x 480
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2,2))(x) #14 x 14 x 480

    #5
    x = Conv2D(960, (3,3),  padding = 'same')(x) #12 x 12 x 960
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((3,3))(x) #6 x 6 x 240
    
    return x


In [10]:
def classification_block_5layer(x):

    x = Dropout(0.2)(x)

    x = Dense(200)(x)
    x = ReLU()(x)
    x = Dropout(0.2)(x)

    x = Dense(50)(x)
    x = ReLU()(x)
    
    x = Dense(1, activation = 'sigmoid', name = 'classification')(x)

    return x


### CNN with Max Global Pooling as flattening layer

In [11]:
inputs = Input(shape = (IMG_SIZE,IMG_SIZE,1))
convolutional_block_x = convolutional_block_5layer(inputs)
fc_block = GlobalMaxPooling2D()(convolutional_block_x) #Global Max Pooling
classification_block_maxpool = classification_block_5layer(fc_block)
l5_maxglobalpool_model = Model(inputs = inputs, outputs = classification_block_maxpool)

### CNN with Average Global Pooling as flattening layer

In [13]:
inputs = Input(shape = (IMG_SIZE,IMG_SIZE,1))
convolutional_block_x = convolutional_block_5layer(inputs)
fc_block = GlobalAveragePooling2D()(convolutional_block_x) #Average Global Pooling
classification_block_avgpool = classification_block_5layer(fc_block)
l5_avgglobalpool_model = Model(inputs = inputs, outputs = classification_block_avgpool)

### CNN with Flatten Layer as flattening layer

In [12]:
inputs = Input(shape = (IMG_SIZE,IMG_SIZE,1))
convolutional_block_x = convolutional_block_5layer(inputs)
fc_block = Flatten()(convolutional_block_x) # Flatten Layer
classification_block_flatten = classification_block_5layer(fc_block)
l5_flatten_model = Model(inputs = inputs, outputs = classification_block_flatten)

In [13]:
# l5_maxglobalpool_model.summary()

###  Compile and Run


In [ ]:
def model_compile_and_fit(model,model_name):

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), \
                  loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), AUC(name='auc')])

    os.makedirs(f"final_models/{model_name}", exist_ok=True)
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=2, min_lr=1e-6,verbose=1)
    
    checkpoint_cb = ModelCheckpoint(filepath=f"final_models/{model_name}/{model_name}_{{epoch:02d}}.keras",save_freq='epoch', \
                                    save_weights_only=True,save_best_only=False,verbose=1)
    
    csv_logger = CSVLogger(f"final_models//{model_name}/{model_name}_log.csv", append=True)
    
    early_stop = EarlyStopping(monitor='val_loss',patience=3,mode='min', verbose = 1)
    
    model.fit(train_ds, validation_data=val_ds, epochs=20, verbose = 1, \
            callbacks=[checkpoint_cb, reduce_lr,csv_logger, early_stop])


In [14]:
# model_compile_and_fit(l5_flatten_model,'l5_flatten')
# model_compile_and_fit(l5_avgglobalpool_model,'l5_avgglobalpool')
model_compile_and_fit(l5_maxglobalpool_model,'L5_max_pool_v2')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.5446 - accuracy: 0.7398 - precision: 0.6171 - recall: 0.4773 - auc: 0.7643
Epoch 1: saving model to final_models/L5_max_pool_v2\L5_max_pool_v2_01.keras
2144/2144 [==============================] - 404s 185ms/step - loss: 0.5446 - accuracy: 0.7398 - precision: 0.6171 - recall: 0.4773 - auc: 0.7643 - val_loss: 0.4504 - val_accuracy: 0.8053 - val_precision: 0.7007 - val_recall: 0.6671 - val_auc: 0.8519 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.4686 - accuracy: 0.7805 - precision: 0.6826 - recall: 0.5780 - auc: 0.8263
Epoch 2: saving model to final_models/L5_max_pool_v2\L5_max_pool_v2_02.keras
2144/2144 [==============================] - 396s 185ms/step - loss: 0.4686 - accuracy: 0.7805 - precision: 0.6826 - recall: 0.5780 - auc: 0.8263 - val_loss: 0.4341 - val_accuracy: 0.8187 - val_precision: 0.7385 - val_recall: 0.6573 - val_auc: 0.8695 - lr: 1.0000e-04
Epoch 3/20
2

# Train 3 Layer Model

In [9]:
#CNN Layers
def CNN_3layer(inputs):
    #1
    x = Conv2D(60, (3,3), padding = 'same')(inputs) #254 x 254 x 30 
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((3,3))(x) #84 x 84 x 60
    
    #2
    x = Conv2D(120, (3,3), padding = 'same')(x) #82 x 82 x 60
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((3,3))(x) #27 x 27 x 60

    #3
    x = Conv2D(240, (3,3), padding = 'same')(x) #25 x 25 x 240
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalMaxPooling2D()(x) #Global Max Pooling
    x = Dropout(0.2)(x)

    x = Dense(100)(x)
    x = ReLU()(x)
    x = Dropout(0.2)(x)

    x = Dense(50)(x)
    x = ReLU()(x)
    
    x = Dense(1, activation = 'sigmoid', name = 'classification')(x)

    return x


In [10]:
inputs = Input(shape = (IMG_SIZE,IMG_SIZE,1))
output_CNN_3layer = CNN_3layer(inputs)
l3_maxpool_model = Model(inputs = inputs, outputs = output_CNN_3layer)

In [12]:
model_compile_and_fit(l3_maxpool_model,'L3_max_pool')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.6195 - accuracy: 0.6832 - precision: 0.5027 - recall: 0.2891 - auc: 0.6620
Epoch 1: saving model to final_models/L3_max_pool\L3_max_pool_01.keras
2144/2144 [==============================] - 176s 79ms/step - loss: 0.6195 - accuracy: 0.6832 - precision: 0.5027 - recall: 0.2891 - auc: 0.6620 - val_loss: 0.5773 - val_accuracy: 0.7142 - val_precision: 0.5427 - val_recall: 0.5910 - val_auc: 0.7532 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.5404 - accuracy: 0.7214 - precision: 0.5908 - recall: 0.4007 - auc: 0.7498
Epoch 2: saving model to final_models/L3_max_pool\L3_max_pool_02.keras
2144/2144 [==============================] - 168s 78ms/step - loss: 0.5404 - accuracy: 0.7214 - precision: 0.5908 - recall: 0.4007 - auc: 0.7498 - val_loss: 0.5721 - val_accuracy: 0.7253 - val_precision: 0.5553 - val_recall: 0.6446 - val_auc: 0.7649 - lr: 1.0000e-04
Epoch 3/20
2144/2144 [====

### 7 Layer CNN model

In [9]:
#CNN Layers
def CNN_7layer(inputs):
    #1
    x = Conv2D(60, (3,3), padding = 'same')(inputs) #254 x 254 x 30 
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2,2))(x) #127 x 127 x 30
    
    #2
    x = Conv2D(120, (3,3), padding = 'same')(x) #125 x 125 x 60
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2,2))(x) #62 x 62 x 60

    #3
    x = Conv2D(240, (3,3), padding = 'same')(x) #60 x 60 x 120
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2,2))(x) #30 x 30 x 120

    #4
    x = Conv2D(480, (3,3),  padding = 'same')(x) #28 x 28 x 240
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2,2))(x) #14 x 14 x 240

    #5
    x = Conv2D(960, (3,3),  padding = 'same')(x) #12 x 12 x 960
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2,2))(x) #6 x 6 x 240

    #6
    x = Conv2D(960, (3,3),  padding = 'same')(x) #10 x 10 x 1940
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((3,3))(x) #5 x 5 x 240

    #7
    x = Conv2D(960, (3,3),  padding = 'same')(x) #3 x 3 x 3380
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalMaxPooling2D()(x) #Global Max Pooling
    x = Dropout(0.2)(x)

    x = Dense(200)(x)
    x = ReLU()(x)
    x = Dropout(0.2)(x)

    x = Dense(50)(x)
    x = ReLU()(x)
    
    x = Dense(1, activation = 'sigmoid', name = 'classification')(x)

    return x


In [10]:
inputs = Input(shape = (IMG_SIZE,IMG_SIZE,1))
output_CNN_7layer = CNN_7layer(inputs)
l7_maxpool_model = Model(inputs = inputs, outputs = output_CNN_7layer)

In [12]:
model_compile_and_fit(l7_maxpool_model,'L7_max_pool')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.4882 - accuracy: 0.7669 - precision: 0.6582 - recall: 0.5543 - auc: 0.8114
Epoch 1: saving model to final_models/L7_max_pool\L7_max_pool_01.keras
2144/2144 [==============================] - 544s 250ms/step - loss: 0.4882 - accuracy: 0.7669 - precision: 0.6582 - recall: 0.5543 - auc: 0.8114 - val_loss: 0.5151 - val_accuracy: 0.7511 - val_precision: 0.7278 - val_recall: 0.3357 - val_auc: 0.7850 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.4440 - accuracy: 0.7964 - precision: 0.7017 - recall: 0.6248 - auc: 0.8479
Epoch 2: saving model to final_models/L7_max_pool\L7_max_pool_02.keras
2144/2144 [==============================] - 533s 248ms/step - loss: 0.4440 - accuracy: 0.7964 - precision: 0.7017 - recall: 0.6248 - auc: 0.8479 - val_loss: 0.4951 - val_accuracy: 0.7613 - val_precision: 0.6340 - val_recall: 0.5740 - val_auc: 0.8072 - lr: 1.0000e-04
Epoch 3/20
2144/2144 [==